In [2]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from jax.nn import softmax
from jax.config import config
from pathlib import Path
from collections import namedtuple

config.update('jax_platform_name', 'cpu')
np.set_printoptions(precision=4)
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams.update({'font.size': 22})

from grl.utils import load_info
from grl.utils.mdp import get_perf
from definitions import ROOT_DIR

In [3]:
results_dir = Path(ROOT_DIR, 'results', 'pomdps_mi_pi_q_abs')


In [4]:
all_results = {}

for results_path in results_dir.iterdir():
    if 'cheese' not in results_path.stem:
        continue
    info = load_info(results_path)
    

In [14]:
agent_path = results_path.parent / 'agents' / f"{results_path.stem}.pkl.npy"

In [22]:
agent = load_info(agent_path)
info['logs']['initial_improvement_policy'], agent.memory, agent.policy

(array([[0.025, 0.925, 0.025, 0.025],
        [0.025, 0.025, 0.925, 0.025],
        [0.025, 0.925, 0.025, 0.025],
        [0.025, 0.925, 0.025, 0.025],
        [0.025, 0.925, 0.025, 0.025],
        [0.925, 0.025, 0.025, 0.025],
        [0.925, 0.025, 0.025, 0.025]]),
 DeviceArray([[[[9.9920e-01, 8.0299e-04],
                [3.8844e-01, 6.1156e-01]],
 
               [[8.4076e-01, 1.5924e-01],
                [6.7398e-01, 3.2602e-01]],
 
               [[9.9244e-01, 7.5613e-03],
                [1.7145e-01, 8.2855e-01]],
 
               [[9.4413e-01, 5.5867e-02],
                [5.5209e-01, 4.4791e-01]],
 
               [[9.8034e-01, 1.9658e-02],
                [9.6175e-01, 3.8252e-02]],
 
               [[9.8364e-01, 1.6364e-02],
                [9.6508e-01, 3.4918e-02]]],
 
 
              [[[9.9528e-01, 4.7183e-03],
                [9.5378e-02, 9.0462e-01]],
 
               [[8.5023e-01, 1.4977e-01],
                [3.4729e-01, 6.5271e-01]],
 
               [[3.1952e-02, 9.68